In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from scipy.spatial import distance

In [6]:
class VRPSDP:
    def __init__(self, costMatrix, demand, pickup, numberOfVehicles, capacityOfVehicle):
        self.costMatrix = costMatrix
        self.n = len(costMatrix)  # Số lượng node (bao gồm depot 0)
        self.demand = demand
        self.pickup = pickup
        self.numberOfVehicles = numberOfVehicles
        self.capacityOfVehicle = capacityOfVehicle
        
        # Các tập hợp chỉ số
        self.V = range(self.n)        # Tập đỉnh V = {0, ..., n-1}
        self.C = range(1, self.n)     # Tập khách hàng C = {1, ..., n-1}
        
        self.initializeLP()

    def initializeLP(self):
        # Khởi tạo mô hình Gurobi
        self.model = gp.Model('VRP-SDP')
        
        # --- 1. Khai báo biến (Variables) ---
        
        # x[i,j]: Biến nhị phân, = 1 nếu đi từ i đến j
        # Mapping với công thức (1) và (11) trong bài báo [cite: 189, 218]
        self.x = self.model.addVars(self.n, self.n, vtype=GRB.BINARY, name="x")

        # ld[i]: Biến liên tục, tải trọng giao hàng còn lại trên xe sau khi thăm i
        # Mapping với định nghĩa biến l_d trong bài báo [cite: 198]
        # Bounds theo công thức (9): d_i <= ld_i <= Q
        self.ld = {}
        for i in self.V:
            self.ld[i] = self.model.addVar(lb=self.demand[i], ub=self.capacityOfVehicle, vtype=GRB.CONTINUOUS, name=f"ld_{i}")

        # l[i]: Biến liên tục, tổng tải trọng trên xe sau khi thăm i
        # Mapping với định nghĩa biến l trong bài báo [cite: 197]
        # Bounds theo công thức (10): p_i <= l_i <= Q
        # Lưu ý: l_i chỉ cần thiết cho tập khách hàng C, nhưng khai báo cả V để tiện index (depot có thể ko dùng)
        self.l = {}
        for i in self.V:
            self.l[i] = self.model.addVar(lb=self.pickup[i], ub=self.capacityOfVehicle, vtype=GRB.CONTINUOUS, name=f"l_{i}")

        # --- 2. Hàm mục tiêu (Objective Function) ---
        
        # Minimize tổng chi phí vận chuyển
        # Mapping với công thức (2) [cite: 201]
        self.model.setObjective(
            gp.quicksum(self.costMatrix[i][j] * self.x[i, j] for i in self.V for j in self.V),
            GRB.MINIMIZE
        )

        # --- 3. Các ràng buộc (Constraints) ---
        
        # Big-M: Theo bài báo mục 4.2, đặt M1 = M2 = Q 
        M = self.capacityOfVehicle

        # (3) & (4): Ràng buộc luồng vào/ra (mỗi khách hàng được thăm đúng 1 lần) [cite: 206]
        for j in self.C:
            self.model.addConstr(gp.quicksum(self.x[i, j] for i in self.V if i != j) == 1, name=f"InDegree_{j}")
            self.model.addConstr(gp.quicksum(self.x[j, i] for i in self.V if i != j) == 1, name=f"OutDegree_{j}")

        # (5): Giới hạn số lượng xe tối đa [cite: 207]
        self.model.addConstr(gp.quicksum(self.x[0, i] for i in self.C) <= self.numberOfVehicles, name="MaxVehicles")

        # (6): Ràng buộc dòng tải trọng giao hàng (Delivery Load Flow) [cite: 208]
        # ld_i >= ld_j + d_i - M * (1 - x_ij)
        for i in self.V:
            for j in self.C:
                if i != j:
                    self.model.addConstr(
                        self.ld[i] >= self.ld[j] + self.demand[i] - M * (1 - self.x[i, j]),
                        name=f"FlowDelivery_{i}_{j}"
                    )

        # (7): Ràng buộc tải trọng xe sau khi thăm khách hàng đầu tiên/bất kỳ (Initial Load Check) [cite: 208]
        # l_j >= ld_j - d_j + p_j
        for j in self.C:
            self.model.addConstr(
                self.l[j] >= self.ld[j] - self.demand[j] + self.pickup[j],
                name=f"LoadConsistency_{j}"
            )

        # (8): Ràng buộc dòng tổng tải trọng (Total Load Flow) [cite: 213]
        # l_j >= l_i - d_j + p_j - M * (1 - x_ij)
        # Áp dụng cho i, j thuộc C (giữa các khách hàng với nhau)
        for i in self.C:
            for j in self.C:
                if i != j:
                    self.model.addConstr(
                        self.l[j] >= self.l[i] - self.demand[j] + self.pickup[j] - M * (1 - self.x[i, j]),
                        name=f"FlowTotalLoad_{i}_{j}"
                    )
        
        # Ràng buộc loại bỏ đi chính mình (x_ii = 0)
        for i in self.V:
            self.model.addConstr(self.x[i, i] == 0)

    def solve(self, time_limit=None):
        """
        Giải mô hình VRPSDP.
        :param time_limit: Giới hạn thời gian giải (tính bằng giây).
        """
        print("Bắt đầu giải bài toán VRPSDP bằng Gurobi...")
        if time_limit:
            self.model.setParam(GRB.Param.TimeLimit, time_limit)
        
        self.model.optimize()

    def getResult(self):
        if self.model.status == GRB.OPTIMAL:
            print("\n" + "="*30)
            print("KẾT QUẢ TỐI ƯU (OPTIMAL SOLUTION)")
            print("="*30)
            print(f"Tổng chi phí (Objective Value): {self.model.objVal}")
            
            # In ra các tuyến đường
            # Logic để trích xuất tuyến đường từ biến x
            active_arcs = []
            for i in self.V:
                for j in self.V:
                    if self.x[i, j].x > 0.9: # Kiểm tra nếu biến xấp xỉ 1
                        active_arcs.append((i, j))
            
            # Reconstruct routes
            current_routes = []
            # Tìm tất cả các điểm bắt đầu từ depot
            start_nodes = [j for (i, j) in active_arcs if i == 0]
            
            for idx, start_node in enumerate(start_nodes):
                route = [0, start_node]
                curr = start_node
                route_load_delivery = 0
                route_load_pickup = 0
                
                while curr != 0:
                    # Tìm điểm tiếp theo
                    next_node = None
                    for (i, j) in active_arcs:
                        if i == curr:
                            next_node = j
                            break
                    
                    if next_node is not None:
                        route.append(next_node)
                        curr = next_node
                    else:
                        break
                
                print(f"Xe {idx + 1}: {' -> '.join(map(str, route))}")
                
            print("="*30)
        elif self.model.status == GRB.INFEASIBLE:
            print("Bài toán không có lời giải (Infeasible). Hãy kiểm tra lại ràng buộc về Capacity hoặc số lượng xe.")
            # Hỗ trợ debug IIS nếu cần
            # self.model.computeIIS()
            # self.model.write("model.ilp")
        else:
            print(f"Không tìm thấy giải pháp tối ưu. Trạng thái Gurobi: {self.model.status}")



In [ ]:
# --- Dữ liệu mẫu từ người dùng ---
costMatrix = [
    [0,9,14,23,32,50,21,49,30,27,35,28,18],
    [9,0,21,22,36,52,24,51,36,37,41,30,20],
    [14,21,0,25,38,5,31,7,36,43,29,7,6],
    [23,22,25,0,42,12,35,17,44,31,31,11,6],
    [32,36,38,42,0,22,37,16,46,37,29,13,14],
    [50,52,5,12,22,0,41,23,10,39,9,17,16],
    [21,24,31,35,37,41,0,26,21,19,10,25,12],
    [49,51,7,17,16,23,26,0,30,28,16,27,12],
    [30,36,36,44,46,10,21,30,0,25,22,10,20],
    [27,37,43,31,37,39,19,28,25,0,20,16,8],
    [35,41,29,31,29,9,10,16,22,20,0,10,10],
    [28,30,7,11,13,17,25,27,10,16,10,0,10],
    [18,20, 6, 6,14,16,12,12,20,8, 10,10,0]
]
demand = [0, 1200, 1700, 1500, 1400, 1700, 1400, 1200, 1900, 1800, 1600, 1700, 1100]
pickup = [0, 0, 1200, 1700, 1500, 1400, 1700, 1400, 1200, 1900, 1800, 1600, 1700]
capacityOfVehicle = 6000
numberOfVehicles = 4

# --- Chạy chương trình ---
lp2 = VRPSDP(costMatrix, demand, pickup, numberOfVehicles, capacityOfVehicle)
lp2.solve(time_limit=60)
lp2.getResult()

Bắt đầu giải bài toán VRPSDP bằng Gurobi...
Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-6300U CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Academic license 2739981 - for non-commercial use only - registered to so___@sis.hust.edu.vn
Optimize a model with 326 rows, 195 columns and 1165 nonzeros (Min)
Model fingerprint: 0x73647520
Model has 156 linear objective coefficients
Variable types: 26 continuous, 169 integer (169 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+03]
  Objective range  [5e+00, 5e+01]
  Bounds range     [1e+00, 6e+03]
  RHS range        [1e+00, 7e+03]
Presolve removed 25 rows and 15 columns
Presolve time: 0.02s
Presolved: 301 rows, 180 columns, 2236 nonzeros
Variable types: 24 continuous, 156 integer (156 binary)

Root relaxation: objective 1.403200e+02, 74 iterations, 0.00 seconds (0.00 work units)

    No

In [ ]:
xCoordinates = [145, 151, 159, 130, 128, 163, 146, 161, 142, 163, 148, 128, 156, 129, 146, 164, 141, 147, 164, 129, 155, 139]
yCoordinates = [215, 164, 261, 254, 252, 247, 246, 242, 239, 236, 232, 231, 217, 214, 208, 208, 206, 193, 193, 189, 185, 182]

costMatrix = np.ndarray(shape=(len(xCoordinates), len(yCoordinates)))
for i in range(len(xCoordinates)):
    for j in range(len(yCoordinates)):
        costMatrix[i][j] = float(distance.euclidean([xCoordinates[i],yCoordinates[i]], [xCoordinates[j],yCoordinates[j]]))

demand = [0, 1100, 700, 800, 1400, 2100, 400, 800, 100, 500, 600, 1200, 1300, 1300, 300, 900, 2100, 1000, 900, 2100, 1000, 900, 2500, 700]
pickup = [0, 0, 1100, 700, 800, 1400, 2100, 400, 800, 100, 500, 600, 1200, 1300, 1300, 300, 900, 2100, 1000, 900, 2100, 1000, 900, 2500]
capacityOfVehicle = 6000
numberOfVehicles = 7
lp2 = VRPSDP(costMatrix, demand, pickup, numberOfVehicles, capacityOfVehicle)
lp2.solve(time_limit=120) # Giới hạn thời gian chạy là 60 giây
lp2.getResult()

Bắt đầu giải bài toán VRPSDP bằng Gurobi...
Set parameter TimeLimit to value 60
Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-6300U CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  60

Academic license 2739981 - for non-commercial use only - registered to so___@sis.hust.edu.vn
Optimize a model with 947 rows, 528 columns and 3550 nonzeros (Min)
Model fingerprint: 0xe23b675c
Model has 462 linear objective coefficients
Variable types: 44 continuous, 484 integer (484 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+03]
  Objective range  [3e+00, 1e+02]
  Bounds range     [1e+00, 6e+03]
  RHS range        [1e+00, 7e+03]
Presolve removed 43 rows and 24 columns
Presolve time: 0.05s
Presolved: 904 rows, 504 columns, 6942 nonzeros
Variable types: 42 continuous, 462 integer (462 binary)

Root relaxation: ob

In [ ]:
xCoordinates = [162, 218, 218, 201, 214, 224, 210, 104, 126, 119, 129, 126, 125, 116, 126, 125, 119, 115, 153, 175, 180, 159, 188, 152, 215, 212, 188, 207, 184, 207]
yCoordinates = [354, 382, 358, 370, 371, 370, 382, 354, 338, 340, 349, 347, 346, 355, 335, 355, 357, 341, 351, 363, 360, 331, 357, 349, 389, 394, 393, 406, 410, 392]

costMatrix = np.ndarray(shape=(len(xCoordinates), len(yCoordinates)))
for i in range(len(xCoordinates)):
    for j in range(len(yCoordinates)):
        costMatrix[i][j] = float(distance.euclidean([xCoordinates[i],yCoordinates[i]], [xCoordinates[j],yCoordinates[j]]))

demand = [0, 300, 3100, 125, 100, 200, 150, 150, 450, 300, 100, 950, 125, 150, 150, 550, 150, 100, 150, 400, 300, 1500, 100, 300, 500, 800, 300, 100, 150, 1000]
pickup = [1000, 0, 300, 3100, 125, 100, 200, 150, 150, 450, 300, 100, 950, 125, 150, 150, 550, 150, 100, 150, 400, 300, 1500, 100, 300, 500, 800, 300, 100, 150]
capacityOfVehicle = 4500
numberOfVehicles = 4

lp2 = VRPSDP(costMatrix, demand, pickup, numberOfVehicles, capacityOfVehicle)
lp2.solve()
lp2.getResult()

In [2]:
class VRPSDP:
    """
    Giải quyết VRPSDP sử dụng Gurobi.
    Implement mô hình Vehicle-flow (Section 4.2) 
    KẾT HỢP VỚI Kỹ thuật mô hình hóa hiệu quả (Section 4.3) của Rieck and Zimmermann (2013).
    """
    def __init__(self, costMatrix, demand, pickup, numberOfVehicles, capacityOfVehicle):
        self.costMatrix = costMatrix
        self.n = len(costMatrix)
        self.nodes = range(self.n)      # V = {0, ..., n}
        self.customers = range(1, self.n) # C = {1, ..., n}
        
        self.demand = demand
        self.pickup = pickup
        self.numberOfVehicles = numberOfVehicles
        self.capacityOfVehicle = capacityOfVehicle
        
        self.model = None
        self.x = None
        self.l = None
        self.ld = None
        
        self._initialize_model()

    def _initialize_model(self):
        self.model = gp.Model('VRPSDP_Enhanced')
        
        # --- 1. Biến quyết định ---
        
        # x[i, j]: Biến nhị phân
        self.x = self.model.addVars(self.nodes, self.nodes, vtype=GRB.BINARY, name="x")

        # [Section 4.3 - Eq 13] Siết chặt cận trên cho biến ld (Tải giao hàng)
        # ld_i <= Q - max(0, p_i - d_i)
        # Ý nghĩa: Nếu tại i nhận hàng nhiều hơn giao, thì tải giao hàng còn lại trên xe không thể đầy 100% Q được.
        ub_ld = {
            i: self.capacityOfVehicle - max(0, self.pickup[i] - self.demand[i]) 
            for i in self.nodes
        }
        self.ld = self.model.addVars(self.nodes, lb=self.demand, ub=ub_ld, vtype=GRB.CONTINUOUS, name="ld")

        # [Section 4.3 - Eq 12] Siết chặt cận trên cho biến l (Tổng tải trọng)
        # l_i <= Q - max(0, d_i - p_i)
        # Ý nghĩa: Nếu tại i giao hàng nhiều hơn nhận, thì tổng tải trọng sau khi rời i không thể đầy 100% Q.
        ub_l = {
            i: self.capacityOfVehicle - max(0, self.demand[i] - self.pickup[i]) 
            for i in self.customers
        }
        self.l = self.model.addVars(self.customers, lb=self.pickup[1:], ub=ub_l, vtype=GRB.CONTINUOUS, name="l")

        # --- 2. Hàm mục tiêu ---
        objective = gp.quicksum(self.costMatrix[i][j] * self.x[i, j] 
                                for i in self.nodes for j in self.nodes if i != j)
        self.model.setObjective(objective, GRB.MINIMIZE)

        # --- 3. Các ràng buộc ---
        
        # x_ii = 0
        self.model.addConstrs((self.x[i, i] == 0 for i in self.nodes), name="no_self_loops")

        # (3) & (4): Luồng vào/ra
        self.model.addConstrs((gp.quicksum(self.x[i, j] for i in self.nodes if i != j) == 1 
                               for j in self.customers), name="indegree")
        self.model.addConstrs((gp.quicksum(self.x[j, i] for i in self.nodes if i != j) == 1 
                               for j in self.customers), name="outdegree")

        # (5): Giới hạn số xe
        self.model.addConstr(gp.quicksum(self.x[0, j] for j in self.customers) <= self.numberOfVehicles, 
                             name="num_vehicles")

        # [Section 4.3 - Modified Eq 6] Delivery Flow với Dynamic Big-M
        # Thay M1 tĩnh bằng M1_ij = Q - max(0, p_j - d_j)
        # Việc tính toán được thực hiện trực tiếp trong generator expression để tối ưu tốc độ
        self.model.addConstrs(
            (self.ld[i] >= self.ld[j] + self.demand[i] - 
             (self.capacityOfVehicle - max(0, self.pickup[j] - self.demand[j])) * (1 - self.x[i, j])
             for i in self.nodes for j in self.customers if i != j), 
            name="delivery_flow_tight"
        )

        # (7): Tải trọng xe tại khách hàng đầu tiên (Giữ nguyên)
        self.model.addConstrs((self.l[j] >= self.ld[j] - self.demand[j] + self.pickup[j] 
                               for j in self.customers), name="load_first_customer")

        # [Section 4.3 - Modified Eq 8] Total Load Flow với Dynamic Big-M
        # Thay M2 tĩnh bằng M2_ij = Q - max(d_j, d_i - p_i + d_j)
        self.model.addConstrs(
            (self.l[j] >= self.l[i] - self.demand[j] + self.pickup[j] - 
             (self.capacityOfVehicle - max(self.demand[j], self.demand[i] - self.pickup[i] + self.demand[j])) * (1 - self.x[i, j])
             for i in self.customers for j in self.customers if i != j), 
            name="load_other_customers_tight"
        )
        
        # Note: Ràng buộc (9) và (10) về Bounds đã được tích hợp ngay khi khai báo biến self.ld và self.l ở trên.

    def solve(self, time_limit=None):
        print("Bắt đầu giải VRPSDP (Enhanced Formulation)...")
        if time_limit:
            self.model.setParam(GRB.Param.TimeLimit, time_limit)
        self.model.optimize()

    def getResult(self):
        if self.model.solCount > 0:
            print("\n" + "="*40)
            print(f"KẾT QUẢ TỐI ƯU (Section 4.3 Model)")
            print(f"Tổng chi phí: {self.model.ObjVal:.2f}")
            print("="*40)
            
            solution_edges = []
            for i in self.nodes:
                for j in self.nodes:
                    if i != j and self.x[i, j].X > 0.5:
                        solution_edges.append((i, j))
            
            starts = [j for i, j in solution_edges if i == 0]
            for idx, start_node in enumerate(starts):
                route = [0, start_node]
                curr = start_node
                cost = self.costMatrix[0][curr]
                load_str = f"(L:{self.l[curr].X:.0f})" # Hiển thị tải trọng tại điểm đầu
                
                while curr != 0:
                    found = False
                    for i, j in solution_edges:
                        if i == curr:
                            route.append(j)
                            cost += self.costMatrix[i][j]
                            if j != 0:
                                load_str += f" -> (L:{self.l[j].X:.0f})"
                            curr = j
                            found = True
                            break
                    if not found: break
                
                print(f"Xe {idx+1}: {' -> '.join(map(str, route))}")
                print(f"       Chi phí: {cost}")
                # In thông tin tải trọng để debug nếu cần
                # print(f"       Tải trọng dọc đường: {load_str}")
            print("="*40)
        elif self.model.status == GRB.INFEASIBLE:
            print("Mô hình không khả thi (Infeasible).")
            # self.model.computeIIS() 
            # self.model.write("infeasible.ilp")
        else:
            print("Không tìm thấy lời giải.")



In [3]:
# --- Dữ liệu mẫu từ người dùng ---
costMatrix = [
    [0,9,14,23,32,50,21,49,30,27,35,28,18],
    [9,0,21,22,36,52,24,51,36,37,41,30,20],
    [14,21,0,25,38,5,31,7,36,43,29,7,6],
    [23,22,25,0,42,12,35,17,44,31,31,11,6],
    [32,36,38,42,0,22,37,16,46,37,29,13,14],
    [50,52,5,12,22,0,41,23,10,39,9,17,16],
    [21,24,31,35,37,41,0,26,21,19,10,25,12],
    [49,51,7,17,16,23,26,0,30,28,16,27,12],
    [30,36,36,44,46,10,21,30,0,25,22,10,20],
    [27,37,43,31,37,39,19,28,25,0,20,16,8],
    [35,41,29,31,29,9,10,16,22,20,0,10,10],
    [28,30,7,11,13,17,25,27,10,16,10,0,10],
    [18,20, 6, 6,14,16,12,12,20,8, 10,10,0]
]
demand = [0, 1200, 1700, 1500, 1400, 1700, 1400, 1200, 1900, 1800, 1600, 1700, 1100]
pickup = [0, 0, 1200, 1700, 1500, 1400, 1700, 1400, 1200, 1900, 1800, 1600, 1700]
capacityOfVehicle = 6000
numberOfVehicles = 4

# --- Chạy chương trình ---
lp2 = VRPSDP(costMatrix, demand, pickup, numberOfVehicles, capacityOfVehicle)
lp2.solve()
lp2.getResult()

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2739981


Academic license 2739981 - for non-commercial use only - registered to so___@sis.hust.edu.vn
Bắt đầu giải VRPSDP (Enhanced Formulation)...
Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-6300U CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Academic license 2739981 - for non-commercial use only - registered to so___@sis.hust.edu.vn
Optimize a model with 326 rows, 194 columns and 1165 nonzeros (Min)
Model fingerprint: 0xa87f5135
Model has 156 linear objective coefficients
Variable types: 25 continuous, 169 integer (169 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+03]
  Objective range  [5e+00, 5e+01]
  Bounds range     [1e+00, 6e+03]
  RHS range        [1e+00, 6e+03]
Presolve removed 25 rows and 14 columns
Presolve time: 0.05s
Presolved: 301 rows, 180 columns, 2236 nonzeros
Variable types: 24 continuous, 156 integer (156 binary)


In [ ]:
xCoordinates = [145, 151, 159, 130, 128, 163, 146, 161, 142, 163, 148, 128, 156, 129, 146, 164, 141, 147, 164, 129, 155, 139]
yCoordinates = [215, 164, 261, 254, 252, 247, 246, 242, 239, 236, 232, 231, 217, 214, 208, 208, 206, 193, 193, 189, 185, 182]

costMatrix = np.ndarray(shape=(len(xCoordinates), len(yCoordinates)))
for i in range(len(xCoordinates)):
    for j in range(len(yCoordinates)):
        costMatrix[i][j] = float(distance.euclidean([xCoordinates[i],yCoordinates[i]], [xCoordinates[j],yCoordinates[j]]))

demand = [0, 1100, 700, 800, 1400, 2100, 400, 800, 100, 500, 600, 1200, 1300, 1300, 300, 900, 2100, 1000, 900, 2100, 1000, 900]
pickup = [0, 0, 1100, 700, 800, 1400, 2100, 400, 800, 100, 500, 600, 1200, 1300, 1300, 300, 900, 2100, 1000, 900, 2100, 1000]
capacityOfVehicle = 6000
numberOfVehicles = 7
lp2 = VRPSDP(costMatrix, demand, pickup, numberOfVehicles, capacityOfVehicle)
lp2.solve() # Giới hạn thời gian chạy là 60 giây
lp2.getResult()

In [4]:
xCoordinates = [162, 218, 218, 201, 214, 224, 210, 104, 126, 119, 129, 126, 125, 116, 126, 125, 119, 115, 153, 175, 180, 159, 188, 152, 215, 212, 188, 207, 184, 207]
yCoordinates = [354, 382, 358, 370, 371, 370, 382, 354, 338, 340, 349, 347, 346, 355, 335, 355, 357, 341, 351, 363, 360, 331, 357, 349, 389, 394, 393, 406, 410, 392]

costMatrix = np.ndarray(shape=(len(xCoordinates), len(yCoordinates)))
for i in range(len(xCoordinates)):
    for j in range(len(yCoordinates)):
        costMatrix[i][j] = float(distance.euclidean([xCoordinates[i],yCoordinates[i]], [xCoordinates[j],yCoordinates[j]]))

demand = [0, 300, 3100, 125, 100, 200, 150, 150, 450, 300, 100, 950, 125, 150, 150, 550, 150, 100, 150, 400, 300, 1500, 100, 300, 500, 800, 300, 100, 150, 1000]
pickup = [0, 0, 300, 3100, 125, 100, 200, 150, 150, 450, 300, 100, 950, 125, 150, 150, 550, 150, 100, 150, 400, 300, 1500, 100, 300, 500, 800, 300, 100, 150]
capacityOfVehicle = 4500
numberOfVehicles = 4

lp2 = VRPSDP(costMatrix, demand, pickup, numberOfVehicles, capacityOfVehicle)
lp2.solve()
lp2.getResult()

Bắt đầu giải VRPSDP (Enhanced Formulation)...
Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-6300U CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Academic license 2739981 - for non-commercial use only - registered to so___@sis.hust.edu.vn
Optimize a model with 1771 rows, 959 columns and 6758 nonzeros (Min)
Model fingerprint: 0xd2208583
Model has 870 linear objective coefficients
Variable types: 59 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 5e+03]
  RHS range        [1e+00, 5e+03]
Presolve removed 73 rows and 38 columns
Presolve time: 0.27s
Presolved: 1698 rows, 921 columns, 13179 nonzeros
Variable types: 58 continuous, 863 integer (863 binary)

Root relaxation: objective 2.817909e+02, 128 iterations, 0.02 seconds (0.00 work units)

